In [1]:
from neural_network.binary_nn import NN
from datasets.sipm_dataset import SiPMDataset
from genetic_algorithm.algorithm import GeneticAlgorithm

In [2]:
# GA parameters
genome_length = 2  # smaller genome for quick test
mutation_prob = 0.05
pop_size = 10       # small population for demo
ngen = 5            # few generations

ga = GeneticAlgorithm(
    nn=NN((2, 8, 2), input=SiPMDataset(),), 
    genome_length=genome_length,
    mutation_prob=mutation_prob,
    pop_size=pop_size,
    ngen=ngen,
    elite_size=2
)

In [3]:
pop, log, hof = ga.run()

Create init population...
Start evolution...
Individual: [1, 1]
Individual: [1, 1]
Individual: [0, 1]
Individual: [1, 1]
Individual: [0, 1]
Individual: [1, 0]
Individual: [1, 1]
Individual: [1, 1]
Individual: [1, 1]
Individual: [1, 1]
gen	nevals	avg	min                 	max                 	diversity	time
0  	10    	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	2 s 
Individual: [1, 1]
Individual: [1, 0]
Individual: [1, 1]
Individual: [1, 0]
Individual: [1, 1]
Individual: [1, 1]
Individual: [0, 1]
Individual: [1, 1]
1  	8     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	4 s 
Individual: [1, 0]
Individual: [1, 1]
Individual: [1, 1]
Individual: [1, 1]
Individual: [1, 1]
Individual: [1, 1]
2  	6     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	5 s 
Individual: [1, 1]
Individual: [1, 1]
Individual: [1, 0]
Individual: [1, 1]
3  	4     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	6 s 
Individual: [1, 1]
Individual: [1, 1]
Individual: [1, 1]
Indivi

In [9]:
print("Best individual:", hof[0])
print("Best fitness:", ga.nn.fitness(hof[0]))
print("GA run complete.\n")
# Example run for the HCL4BNN package
print(f"log: \n{log}")
print(f"pop: \n{pop}")

Best individual: [0, 1]
Individual: [0, 1]
Best fitness: 800
GA run complete.

log: 
gen	nevals	avg	min                 	max                 	diversity	time
0  	10    	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	2 s 
1  	8     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	4 s 
2  	6     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	5 s 
3  	4     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	6 s 
4  	6     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	8 s 
5  	8     	800	(np.float64(800.0),)	(np.float64(800.0),)	0.1      	9 s 
pop: 
[[1, 1], [1, 1], [1, 0], [1, 1], [1, 1], [1, 1], [1, 0], [1, 1], [1, 1], [1, 1]]


# Testing

In [4]:
nn = NN((2, 8, 2), input=SiPMDataset())
x, y = SiPMDataset().load_data()
print(f"Inputs (X): \n{x}\n\n\nLabels (y): \n{y}\n\n\n")

Inputs (X): 
[[ 8  8 33 ...  8  8  8]
 [ 8  8 32 ...  8  8  8]
 [ 8  8 19 ...  8  8  8]
 ...
 [ 8  8 15 ...  8  8  8]
 [ 8  8 20 ...  8  8  8]
 [ 8  8 34 ...  8  8  8]]


Labels (y): 
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]



